In [2]:
using Revise

using RigidBodyDynamics
using RigidBodyDynamics: Bounds
using LCPSim
# using Ipopt
using ConditionalJuMP
using Gurobi
using RigidBodyTreeInspector
using DrakeVisualizer

INFO: Recompiling stale cache file /Users/blandry/.julia/lib/v0.6/LCPSim.ji for module LCPSim.


HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/8c8a043c2f00369f3e7ae550277d2a54e59a34ad-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/8c8a043c2f00369f3e7ae550277d2a54e59a34ad-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [3]:
urdf = joinpath("..","urdf", "brick.urdf")
mechanism = parse_urdf(Float64, urdf)

body = findbody(mechanism, "brick")
basejoint = joint_to_parent(body, mechanism)
floatingjoint = Joint(basejoint.name, frame_before(basejoint), frame_after(basejoint), QuaternionFloating{Float64}())
replace_joint!(mechanism, basejoint, floatingjoint)

position_bounds(floatingjoint) .= Bounds(-100, 100)
velocity_bounds(floatingjoint) .= Bounds(-100, 100);

In [35]:
x0 = MechanismState(mechanism)
set_configuration!(x0, [1., 0., 0., 0., 0., 0., 2.])
set_velocity!(x0, [0., 0., 0., 0., 0. , 0.])
env = LCPSim.parse_contacts(mechanism, urdf);
controller = LCPSim.passive_controller()
Δt = 0.01
N = 100

100

In [11]:
q = [1., 0., 0., 0., 0., 0., 2.]
v = [0., 0., 0., 0., 0. , 0.]
u = [0., 0., 0., 0., 0. , 0.]
eval_g, eval_jac_g = update_constraints_implicit(env,mechanism,Δt,q,v,u)

(LCPSim.eval_g, LCPSim.eval_jac_g)

In [13]:
x = rand(length(q)+length(v))
g = zeros(length(x)+length(env.contacts))
eval_g(x, g)
values = zeros(length(g)*length(x))
eval_jac_g(x, :Eval, [], [], values)

221-element Array{Float64,1}:
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 1.0        
 0.0        
 0.0        
 0.0        
 2.00195e-12
 0.0        
 ⋮          
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        

In [36]:
results = LCPSim.simulate(x0, controller, env, Δt, N)

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      169
Number of nonzeros in inequality constraint Jacobian.:       52
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       13
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       13
                     variables with only upper bounds:        0
Total number of equality constraints.................:       13
Total number of inequality constraints...............:        4
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        4
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

13×101 Array{Float64,2}:
 1.0   1.0       1.0       1.0       1.0      …   1.0       1.0       1.0    
 0.0   0.0       0.0       0.0       0.0          0.0       0.0       0.0    
 0.0   0.0       0.0       0.0       0.0          0.0       0.0       0.0    
 0.0   0.0       0.0       0.0       0.0          0.0       0.0       0.0    
 0.0   0.0       0.0       0.0       0.0          0.0       0.0       0.0    
 0.0   0.0       0.0       0.0       0.0      …   0.0       0.0       0.0    
 2.0   1.99902   1.99706   1.99411   1.99019     -2.75883  -2.85595  -2.95405
 0.0   0.0       0.0       0.0       0.0          0.0       0.0       0.0    
 0.0   0.0       0.0       0.0       0.0          0.0       0.0       0.0    
 0.0   0.0       0.0       0.0       0.0          0.0       0.0       0.0    
 0.0   0.0       0.0       0.0       0.0      …   0.0       0.0       0.0    
 0.0   0.0       0.0       0.0       0.0          0.0       0.0       0.0    
 0.0  -0.0981   -0.1962   -0.2943   -0.

In [19]:
qs = [configuration(r.state) for r in results]
ts = cumsum([r.Δt for r in results]);
for q in qs
    println(q)
end
# println(qs)
# plot([q[7] for q in qs])

LoadError: [91mtype Float64 has no field state[39m

In [20]:
results

13×11 Array{Float64,2}:
 1.0   1.0       1.0       1.0       1.0      …   1.0       1.0       1.0    
 0.0   0.0       0.0       0.0       0.0          0.0       0.0       0.0    
 0.0   0.0       0.0       0.0       0.0          0.0       0.0       0.0    
 0.0   0.0       0.0       0.0       0.0          0.0       0.0       0.0    
 0.0   0.0       0.0       0.0       0.0          0.0       0.0       0.0    
 0.0   0.0       0.0       0.0       0.0      …   0.0       0.0       0.0    
 2.0   1.99902   1.99706   1.99411   1.99019      1.96468   1.95586   1.94605
 0.0   0.0       0.0       0.0       0.0          0.0       0.0       0.0    
 0.0   0.0       0.0       0.0       0.0          0.0       0.0       0.0    
 0.0   0.0       0.0       0.0       0.0          0.0       0.0       0.0    
 0.0   0.0       0.0       0.0       0.0      …   0.0       0.0       0.0    
 0.0   0.0       0.0       0.0       0.0          0.0       0.0       0.0    
 0.0  -0.0981   -0.1962   -0.2943   -0.3

In [37]:
qs = [results[1:num_positions(x0),i] for i in 1:N]
ts = cumsum([Δt for i in 1:N]);

In [28]:
qs

10-element Array{Array{Float64,1},1}:
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0]    
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.99902]
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.99706]
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.99411]
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.99019]
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.98529]
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9794] 
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.97253]
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.96468]
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.95586]

In [22]:
DrakeVisualizer.any_open_windows() || (DrakeVisualizer.new_window(); sleep(1));
geometries = visual_elements(mechanism, URDFVisuals(urdf))
vis = Visualizer(mechanism, geometries);

In [38]:
RigidBodyTreeInspector.animate(vis, mechanism, ts, qs; realtimerate = .5);